In [27]:
import pandas as pd
import requests
import json

In [28]:
#create list of zipcodes from txt file with zips
zips = open('chosen_zips.txt', 'r').readlines()
zips = [z.replace('\n', '') for z in zips]
zips = ','.join(zips)

In [29]:
#census api key
apiKey = ""

In [30]:
#same thing with variables
variables = open('zbp_variables.txt', 'r').readlines()
variables = [z.replace('\n', '') for z in variables]
variables = ','.join(variables)

In [31]:
#just to check for errors, if a ',' appears at the end you need to delete the last blank line in the txt file
print(variables)
#the first variable here corresponds to first value in columnNames

ESTAB


In [32]:
columnNames = ['estab', 'zipcode']

In [33]:
#i really should write a loop for this instead of copying it three times

In [34]:
#base api, this is where you change the year and dataset
#it could be cleaned a little more with %s
#note it is currently the 2000 zbp dataset
baseAPI = "https://api.census.gov/data/2000/zbp?key=%s&get=%s&for=zipcode:%s"
#custom API constructed
calledAPI = baseAPI % (apiKey, variables, zips)
#call API and collect the response
response = requests.get(calledAPI)
#check if all the variables are valid, if ok it will just print, if not there will be an error
#print(response.text)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#store the response in a dataframe
df00 = pd.DataFrame(columns=columnNames, data=formattedResponse)

In [35]:
#base api, this is where you change the year and dataset
#it could be cleaned a little more with %s
#note it is currently the 2010 zbp dataset
baseAPI = "https://api.census.gov/data/2010/zbp?key=%s&get=%s&for=zipcode:%s"
#custom API constructed
calledAPI = baseAPI % (apiKey, variables, zips)
#call API and collect the response
response = requests.get(calledAPI)
#check if all the variables are valid, if ok it will just print, if not there will be an error
#print(response.text)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#store the response in a dataframe
df10 = pd.DataFrame(columns=columnNames, data=formattedResponse)

In [36]:
#base api, this is where you change the year and dataset
#it could be cleaned a little more with %s
#note it is currently the 2017 zbp dataset
baseAPI = "https://api.census.gov/data/2017/zbp?key=%s&get=%s&for=zipcode:%s"
#custom API constructed
calledAPI = baseAPI % (apiKey, variables, zips)
#call API and collect the response
response = requests.get(calledAPI)
#check if all the variables are valid, if ok it will just print, if not there will be an error
#print(response.text)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#store the response in a dataframe
df17 = pd.DataFrame(columns=columnNames, data=formattedResponse)

In [37]:
#now add a year column
df00['year'] = '2000'
df10['year'] = '2010'
df17['year'] = '2017'


In [38]:
#add the town_names df
#fixed the dropping leading zero problem
name_df = pd.read_excel("town_names1.xlsx", dtype={'zipcode': object})


In [39]:
name_df.head()

,zipcode,territory,town,county
0,07059,Warren,Warren,Somerset
1,07060,Warren,Plainfield,Union
2,07062,Warren,Plainfield,Union
3,07063,Warren,Plainfield,Union
4,07069,Warren,Watchung,Somerset


In [40]:
df00 = df00.set_index('zipcode').join(name_df.set_index('zipcode'))
df10 = df10.set_index('zipcode').join(name_df.set_index('zipcode'))
df17 = df17.set_index('zipcode').join(name_df.set_index('zipcode'))
#concat combines them by rows
df_main = pd.concat([df00, df10, df17])
#panel data format, year1 year2 year1 year 2
df_main.sort_values(by=['zipcode', 'year'], inplace=True)

In [41]:
df_main.head()

,estab,year,territory,town,county
zipcode,,,,,
07059,634,2000,Warren,Warren,Somerset
07059,682,2010,Warren,Warren,Somerset
07059,744,2017,Warren,Warren,Somerset
07060,872,2000,Warren,Plainfield,Union
07060,830,2010,Warren,Plainfield,Union


In [42]:
#move some columns around so it will be easier to read in excel
#remember zipcode is the index not a column
df_main = df_main[['territory', 'town', 'county', 'year', 'estab']]


In [43]:
#trying to find percent change
#because zipcode is our key this is a problem, there are multiple zips for one town
#df_main = df_main.groupby(['zipcode'])(['estab']).pct_change()

#make a new df with reset index
dfmod = df_main.reset_index()

In [44]:
dfmod.head()

,zipcode,territory,town,county,year,estab
0,07059,Warren,Warren,Somerset,2000,634
1,07059,Warren,Warren,Somerset,2010,682
2,07059,Warren,Warren,Somerset,2017,744
3,07060,Warren,Plainfield,Union,2000,872
4,07060,Warren,Plainfield,Union,2010,830


In [45]:
dfmod['estab'] = dfmod['estab'].apply(pd.to_numeric, errors='coerce')
dfmod['p_change'] = dfmod.groupby(['zipcode'])['estab'].pct_change()


In [46]:
dfmod.set_index(['zipcode'], inplace=True)

In [48]:
dfmod.to_excel('zbp.xlsx')